In [62]:
# Imports
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import torch
import torch.nn as nn
from pytorch_forecasting.models import TemporalFusionTransformer






In [63]:
# Load SP500 and US macroeconomic data from CSV files
data = pd.read_csv('./data/SP500_data/GOOGL.csv', parse_dates=['Date'])
macro_data = pd.read_csv('./data/SP500_data/US_macroeconomics.csv', parse_dates=['date'])

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)  # Set the date column as the index if not already set

# Merge datasets on the date field
merged_data = pd.merge(data, macro_data, left_on='Date', right_on='date', how='outer')

# Handle missing values: forward fill then backward fill to cover all gaps
merged_data.interpolate(method='ffill', inplace=True)
merged_data.interpolate(method='bfill', inplace=True)

# Drop the extra date column to avoid duplication
merged_data.drop(columns=['date'], inplace=True)

print(merged_data)



             Open         High          Low        Close    Adj Close  \
0       50.050049    52.082081    48.028027    50.220219    50.220219   
1       50.050049    52.082081    48.028027    50.220219    50.220219   
2       50.050049    52.082081    48.028027    50.220219    50.220219   
3       50.050049    52.082081    48.028027    50.220219    50.220219   
4       50.050049    52.082081    48.028027    50.220219    50.220219   
...           ...          ...          ...          ...          ...   
4787  2722.030029  2821.000000  2722.030029  2797.360107  2797.360107   
4788  2774.050049  2791.770020  2756.699951  2765.510010  2765.510010   
4789  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4790  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4791  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   

          Volume      CPI  Mortgage_rate  Unemp_rate        NASDAQ  \
0     44659096.0   85.600        14.2050         7.5 

/var/folders/d9/x_bnq71n58lbk4_znq_twz6h0000gn/T/ipykernel_3878/1542437413.py:12: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.interpolate(method='ffill', inplace=True)
/var/folders/d9/x_bnq71n58lbk4_znq_twz6h0000gn/T/ipykernel_3878/1542437413.py:13: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.interpolate(method='bfill', inplace=True)


In [64]:
# Limit the dataset to the first 1000 rows
limited_data = merged_data.head(1000)

# Split the limited data into train and test sets
train_data = limited_data[:int(0.8*len(limited_data))]
test_data = limited_data[int(0.8*len(limited_data)):]

# Display the shapes of the train and test data
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (800, 13)
Test data shape: (200, 13)


In [65]:
from torch.utils.data import Dataset

# Define input columns and specify which are categorical and which are continuous
input_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "CPI", "Mortgage_rate", "Unemp_rate", "NASDAQ", "disposable_income", "Personal_consumption_expenditure", "personal_savings"]
continuous_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]  # Example continuous columns
categorical_columns = ["CPI", "Mortgage_rate", "Unemp_rate", "NASDAQ", "disposable_income", "Personal_consumption_expenditure", "personal_savings"]  # Example categorical columns
target_column = "Close"  # Assuming you want to predict the closing price

class StockPriceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x_cat = self.data.loc[idx, categorical_columns].values
        x_cont = self.data.loc[idx, continuous_columns].values
        y = self.data.loc[idx, target_column]
        # Assuming all sequences are of the same length, which might need adjustment based on your actual data
        encoder_length = len(x_cat)  # or some other logic to determine the actual length
        decoder_length = len(x_cat)  # similarly, adjust as necessary

        return {
            "encoder_cat": torch.tensor(x_cat, dtype=torch.float32).unsqueeze(0),
            "encoder_cont": torch.tensor(x_cont, dtype=torch.float32).unsqueeze(0),
            "decoder_cat": torch.tensor(x_cat, dtype=torch.float32).unsqueeze(0),
            "decoder_cont": torch.tensor(x_cont, dtype=torch.float32).unsqueeze(0),
            "encoder_lengths": torch.tensor([encoder_length], dtype=torch.long),
            "decoder_lengths": torch.tensor([decoder_length], dtype=torch.long),
            "target": torch.tensor(y, dtype=torch.float32).unsqueeze(0),
        }

train_dataset = StockPriceDataset(train_data)
test_dataset = StockPriceDataset(test_data)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(train_dataset)

In [75]:
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule



# Define the TFT model
class StockPriceTFT(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(ignore=['loss', 'logging_metrics'])
        self.tft = TemporalFusionTransformer(
            hidden_size=config["hidden_size"],
            lstm_layers=config["lstm_layers"],
            dropout=config["dropout"],
            output_size=config["output_size"],
            loss=config["loss"],
            attention_head_size=config["attention_head_size"],
            max_encoder_length=config["max_encoder_length"],
            static_categoricals=config["static_categoricals"],
            static_reals=config["static_reals"],
            time_varying_categoricals_encoder=config["time_varying_categoricals_encoder"],
            time_varying_categoricals_decoder=config["time_varying_categoricals_decoder"],
            categorical_groups=config["categorical_groups"],
            time_varying_reals_encoder=config["time_varying_reals_encoder"],
            time_varying_reals_decoder=config["time_varying_reals_decoder"],
            x_reals=config["x_reals"],
            x_categoricals=config["x_categoricals"],
            hidden_continuous_size=config["hidden_continuous_size"],
            hidden_continuous_sizes=config["hidden_continuous_sizes"],
            embedding_sizes=config["embedding_sizes"],
            embedding_paddings=config["embedding_paddings"],
            embedding_labels=config["embedding_labels"],
            learning_rate=config["learning_rate"],
            log_interval=config["log_interval"],
            log_val_interval=config["log_val_interval"],
            log_gradient_flow=config["log_gradient_flow"],
            reduce_on_plateau_patience=config["reduce_on_plateau_patience"],
            monotone_constaints=config["monotone_constaints"],
            share_single_variable_networks=config["share_single_variable_networks"],
            causal_attention=config["causal_attention"],
            logging_metrics=config["logging_metrics"]
        )

    def forward(self, x):
        # Example debug statements before a problematic layer
        print("Before layer X, shape of input:", x["encoder_cont"].shape)
        # layer operations
        output = self.layer_x(x["encoder_cont"])
        print("After layer X, output shape:", output.shape)
        # more operations
        return output
# Configure the TFT model
config = {
    "hidden_size": 64,
    "lstm_layers": 2,
    "dropout": 0.1,
    "output_size": 1,
    "loss": MeanSquaredError(),
    "attention_head_size": 4,
    "max_encoder_length": 30,
    "static_categoricals": [],
    "static_reals": [],
    "time_varying_categoricals_encoder": [],
    "time_varying_categoricals_decoder": [],
    "categorical_groups": {},
    "time_varying_reals_encoder": ["price"],
    "time_varying_reals_decoder": ["price"],
    "x_reals": ["price"],
    "x_categoricals": [],
    "hidden_continuous_size": 16,
    "hidden_continuous_sizes": {},
    "embedding_sizes": {},
    "embedding_paddings": [],
    "embedding_labels": {},
    "learning_rate": 0.001,
    "log_interval": 10,
    "log_val_interval": 10,
    "log_gradient_flow": False,
    "reduce_on_plateau_patience": 10,
    "monotone_constaints": {},
    "share_single_variable_networks": False,
    "causal_attention": True,
    "logging_metrics": None
}

# Create the TFT model
model = StockPriceTFT(config)

# Prepare your data

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
num_epochs = 100


for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_loader:
        print("Batch keys:", batch.keys())
        print("Shape of encoder_cat:", batch["encoder_cat"].shape)
        print("Shape of encoder_cont:", batch["encoder_cont"].shape)
        print("Shape of decoder_cat:", batch["decoder_cat"].shape)
        print("Shape of decoder_cont:", batch["decoder_cont"].shape)
        print("Shape of target:", batch["target"].shape)
    


        optimizer.zero_grad()
        output = model({
            "encoder_cat": batch["encoder_cat"].squeeze(1),
            "encoder_cont": batch["encoder_cont"].squeeze(1),
            "decoder_cat": batch["decoder_cat"].squeeze(1),
            "decoder_cont": batch["decoder_cont"].squeeze(1),
            "encoder_lengths": batch["encoder_lengths"].squeeze(1),
            "decoder_lengths": batch["decoder_lengths"].squeeze(1)
        })
        loss = config["loss"](output, batch["target"].squeeze(1))
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in test_loader:
            output = model(batch["x"])
            val_loss += config["loss"](output, batch["target"]).item()
        val_loss /= len(test_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}")
# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_data)

NameError: name 'LightningModule' is not defined